In [3]:
import pandas as pd
import numpy as np

Lectura de dades

In [28]:
activitats_v = pd.read_csv("data/activitats_valencia.csv")
activitats_a = pd.read_csv("data/activitats_alacant.csv")
activitats_c = pd.read_csv("data/activitats_castello.csv")
ajuntaments_info = pd.read_csv("data/ajuntaments_info.csv")

* Seleccio de variables i canvi de nom

In [27]:
columns = ["Marca temporal", "Localitat", "Data", "Categoria", "En cas d'altres, quina?", "Nom de l'activitat ", "Companyia, artista", "Lloc", "Llengua de l'activitat", "Preu", "Regidoria organitzadora", "Llengua de la publicitat", "Font",  "Persona que ha introduït les dades"]
columns_v = ["Marca de temps", "Localitat", "Data", "Categoria", "En cas d'altres, quina?", "Nom de l'activitat ", "Companyia, artista", "Lloc", "Llengua de l'activitat", "Preu", "Regidoria organitzadora", "Llengua de la publicitat", "Font",  "Persona que ha introduït les dades"]
column_rename = ["marca_temps", "localitat", "data", "categoria", "categoria_alternativa", "nom_activitat", "companyia", "lloc", "llengua_activitat", "preu", "regidoria", "llengua_publicitat", "font", "persona_dades"]

In [29]:
activitats_v = activitats_v[columns_v]
activitats_v.columns = column_rename
activitats_c = activitats_c[columns]
activitats_c.columns = column_rename
activitats_a = activitats_a[columns]
activitats_a.columns = column_rename

* Juntem els 3 datasets

In [34]:
activitats = pd.concat([activitats_a, activitats_v, activitats_c], axis = 0)

In [35]:
activitats.shape

(6899, 14)

* Preprocessament de dades

* Creació de noves variables

Preu_mitjà

In [69]:
activitats["preu_mitja"] = activitats.preu.str.findall(r"\d+[,'.]?\d*")
activitats["preu_mitja"].loc[activitats.preu.notnull()] = activitats.loc[activitats.preu.notnull()].preu_mitja.apply(lambda x: np.mean([float(num.replace("'", ".").replace(",", ".")) for num in x]))

c:\Users\quimm\Anaconda3\envs\venv\lib\site-packages\numpy\core\fromnumeric.py:3440: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,


Categoria New

In [86]:
type("d") == str

True

In [88]:
def assign_category(category_alternative):

    if type(category_alternative) != str:
        category_new = np.nan
    else:
        category_alternative = category_alternative.lower()
        if "fall" in category_alternative:
            category_new = "Falles"
        elif "màg" in category_alternative:
            category_new = "Màgia"
        elif "visita" in category_alternative:
            category_new = "Activitats sobre patrimoni"
        elif "circ" in category_alternative:
            category_new = "Circ"
        elif "concu" in category_alternative:
            category_new = "Concurs"
        elif "premi" in category_alternative:
            category_new = "Premis"
        elif "taller" in category_alternative:
            category_new = "Taller"
        elif "espectac" in category_alternative:
            category_new = "Espectacle"
        else:
            category_new = np.nan

    return category_new


In [92]:
activitats["categoria_new"] = activitats["categoria_alternativa"].apply(assign_category)
activitats["categoria_new"] = np.where(activitats['categoria_new'].isnull(), activitats['categoria'], activitats['categoria_new'])

* Agrupacions

llengua activitat

In [153]:
grup1 = activitats.groupby(["localitat","llengua_activitat"])["llengua_activitat"].count()
grup1 = pd.DataFrame(grup1)
grup1.columns = ["count_llengua"]
grup1= grup1.reset_index()

grup1 = grup1.groupby("localitat").apply(lambda x : x.sort_values(by = 'count_llengua', ascending = False).head(3).reset_index(drop = True))
grup1.columns = ["localitat_col", 'llengua_activitat', 'count_llengua']

llengua_activitat_top1 = grup1.groupby("localitat").apply(lambda x : x.head(1).reset_index(drop = True))["llengua_activitat"]
llengua_activitat_top2 = grup1.groupby("localitat").apply(lambda x : x.head(2).tail(1).reset_index(drop = True))["llengua_activitat"]
llengua_activitat_top3 = grup1.groupby("localitat").apply(lambda x : x.tail(1).reset_index(drop = True))["llengua_activitat"]

llengua publicitat

In [150]:
grup1 = activitats.groupby(["localitat","llengua_publicitat"])["llengua_publicitat"].count()
grup1 = pd.DataFrame(grup1)
grup1.columns = ["count_llengua"]
grup1= grup1.reset_index()

grup1 = grup1.groupby("localitat").apply(lambda x : x.sort_values(by = 'count_llengua', ascending = False).head(2).reset_index(drop = True))
grup1.columns = ["localitat_col", 'llengua_activitat', 'count_llengua']

llengua_publicitat_top1 = grup1.groupby("localitat").apply(lambda x : x.head(1).reset_index(drop = True))["llengua_activitat"]
llengua_publicitat_top2 = grup1.groupby("localitat").apply(lambda x : x.tail(1).reset_index(drop = True))["llengua_activitat"]

Language position

In [155]:
grup1 = activitats.groupby(["localitat","llengua_activitat"])["llengua_activitat"].count()
grup1 = pd.DataFrame(grup1)
grup1.columns = ["count_llengua"]
grup1= grup1.reset_index()

grup1 = grup1.groupby("localitat").apply(lambda x : x.sort_values(by = 'count_llengua', ascending = False).reset_index(drop = True))
grup1.columns = ["localitat_col", 'llengua_activitat', 'count_llengua']


In [223]:
grup1.groupby("localitat").apply(lambda x : x.where(x["llengua_activitat"]=="Valencià/català"))

localitat_col llengua_activitat  count_llengua
localitat                                                 
Alacant   0           NaN               NaN            NaN
          1           NaN               NaN            NaN
          2           NaN               NaN            NaN
          3       Alacant   Valencià/català            2.0
          4           NaN               NaN            NaN
...                   ...               ...            ...
Xàbia     5           NaN               NaN            NaN
Xàtiva    0        Xàtiva   Valencià/català           25.0
          1           NaN               NaN            NaN
          2           NaN               NaN            NaN
          3           NaN               NaN            NaN

[557 rows x 3 columns]

In [173]:
pos_val = grup1.groupby("localitat").apply(lambda x : x.index[x["llengua_activitat"]=="Valencià/català"])

In [224]:
pos_val

,0
localitat,
Alacant,"MultiIndex([('Alacant', 3)], names=..."
Alaquàs,"MultiIndex([('Alaquàs', 2)], names=..."
Albal,"MultiIndex([('Albal', 1)], names=['..."
Albatera,"MultiIndex([], names=['localitat', None])"
Alberic,"MultiIndex([('Alberic', 0)], names=..."
...,...
Vinaròs,"MultiIndex([('Vinaròs', 1)], names=..."
Xirivella,"MultiIndex([('Xirivella', 3)], name..."
Xixona,"MultiIndex([('Xixona', 1)], names=[..."


In [212]:
[x[0].get_level_values(None) for x in pos_val.values]

AttributeError: 'Int64Index' object has no attribute 'reset_index'

In [221]:
pos_val.values[0][0].get_level_values(None).index

AttributeError: 'Int64Index' object has no attribute 'index'

In [196]:
pos_val = [val[0][0][0] for val in ]

IndexError: index 0 is out of bounds for axis 0 with size 0

Creating new dataset

In [233]:
activitats_summaries = pd.DataFrame(llengua_activitat_top1)
activitats_summaries = activitats_summaries.reset_index()
del activitats_summaries["level_1"]
activitats_summaries.columns = ['localitat', 'llengua_activitat_top1']

In [243]:
activitats_summaries['llengua_activitat_top2'] = llengua_activitat_top2.values
activitats_summaries['llengua_activitat_top3'] = llengua_activitat_top3.values
activitats_summaries['llengua_publicitat_top1'] = llengua_publicitat_top1.values
activitats_summaries['llengua_publicitat_top2'] = llengua_publicitat_top2.values

In [245]:
activitats_summaries

,localitat,llengua_activitat_top1,llengua_activitat_top2,llengua_activitat_top3,llengua_publicitat_top1,llengua_publicitat_top2
0,Alacant,Espanyol,Activitat sense llengua,No hi ha informació,Espanyol,Bilingüe
1,Alaquàs,Espanyol,Activitat sense llengua,Valencià/català,Bilingüe,Valencià/català
2,Albal,Activitat sense llengua,Valencià/català,Espanyol,Valencià/català,Espanyol
3,Albatera,Espanyol,Activitat sense llengua,Activitat sense llengua,Espanyol,Espanyol
4,Alberic,Valencià/català,Activitat sense llengua,Espanyol,Valencià/català,Valencià/català
...,...,...,...,...,...,...
132,Vinaròs,Activitat sense llengua,Valencià/català,Espanyol,Valencià/català,Bilingüe
133,Xirivella,Activitat sense llengua,Espanyol,No hi ha informació,Bilingüe,Valencià/català
134,Xixona,Activitat sense llengua,Valencià/català,Espanyol,Valencià/català,Espanyol
135,Xàbia,Activitat sense llengua,Valencià/català,Espanyol,Bilingüe,Valencià/català


Saving Datasets

In [247]:
activitats.to_csv("data/activitats.csv")

In [248]:
activitats_summaries.to_csv("data/activitats_summaries.csv")